In [1]:
import os  
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1" 

import keras
from keras.datasets import cifar10
from keras import backend as K
from keras.layers import Input, Conv2D,GlobalAveragePooling2D, Dense, BatchNormalization, Activation
from keras.models import Model
from keras.layers import DepthwiseConv2D

from keras import optimizers,regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import he_normal
from keras.callbacks import LearningRateScheduler, TensorBoard, ModelCheckpoint

num_classes        = 10
batch_size         = 64         # 64 or 32 or other
epochs             = 300
iterations         = 782       
USE_BN=True
DROPOUT=0.2 # keep 80%
CONCAT_AXIS=3
weight_decay=1e-4
DATA_FORMAT='channels_last' # Theano:'channels_first' Tensorflow:'channels_last'
alpha = 0.5

log_filepath  = './mobilenet_slim_n_0.5'

Using TensorFlow backend.


In [2]:
def color_preprocessing(x_train,x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    mean = [125.307, 122.95, 113.865]
    std  = [62.9932, 62.0887, 66.7048]
    for i in range(3):
        x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
        x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]
    return x_train, x_test

def scheduler(epoch):
    if epoch < 100:
        return 0.01
    if epoch < 200:
        return 0.001
    return 0.0001

# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test  = keras.utils.to_categorical(y_test, num_classes)
x_train, x_test = color_preprocessing(x_train, x_test)

In [3]:
def depthwise_separable(x,params):
    # f1/f2 filter size, s1 stride of conv
    (s1,f2) = params
    x = DepthwiseConv2D((3,3),strides=(s1[0],s1[0]), padding='same',
                        depthwise_initializer="he_normal",depthwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = Conv2D(int(alpha*f2[0]), (1,1), strides=(1,1), padding='same',
               kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    return x

In [4]:
def MobileNet(img_input,shallow=False, classes=10):
    """Instantiates the MobileNet.Network has two hyper-parameters
        which are the width of network (controlled by alpha)
        and input size.
        # Arguments
            alpha: optional parameter of the network to change the 
                width of model.
            shallow: optional parameter for making network smaller.
            classes: optional number of classes to classify images
                into.
    """
    # change stride
    x = Conv2D(int(alpha*32), (3,3), strides=(1,1), padding='same',
              kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(weight_decay))(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = depthwise_separable(x,params=[(1,),(64,)])
    x = depthwise_separable(x,params=[(1,),(128,)])# change stride
    x = depthwise_separable(x,params=[(1,),(128,)])
    x = depthwise_separable(x,params=[(1,),(256,)])# change stride
    x = depthwise_separable(x,params=[(1,),(256,)])
    x = depthwise_separable(x,params=[(2,),(512,)])
    if not shallow:
        for _ in range(5):
            x = depthwise_separable(x,params=[(1,),(512,)])
            
    x = depthwise_separable(x,params=[(2,),(1024,)])
    x = depthwise_separable(x,params=[(1,),(1024,)])

    x = GlobalAveragePooling2D()(x)
    out = Dense(classes, activation='softmax')(x)
    return out

In [5]:
img_input=Input(shape=(32,32,3))
output = MobileNet(img_input)
model=Model(img_input,output)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 16)        448       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 16)        64        
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 16)        0         
_________________________________________________________________
depthwise_conv2d_1 (Depthwis (None, 32, 32, 16)        160       
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 16)        64        
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 16)        0         
__________

In [6]:
# set optimizer
sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# set callback
tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=0)
change_lr = LearningRateScheduler(scheduler)
cbks = [change_lr,tb_cb]

# set data augmentation
datagen = ImageDataGenerator(horizontal_flip=True,
                             width_shift_range=0.125,
                             height_shift_range=0.125,
                             fill_mode='constant',cval=0.)
datagen.fit(x_train)

# start training
model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                    steps_per_epoch=iterations,
                    epochs=epochs,
                    callbacks=cbks,
                    validation_data=(x_test, y_test))
model.save('mobilenet_slim_n_0.5.h5')

Epoch 1/300
782/782 [==============================] - 57s 73ms/step - loss: 2.1583 - acc: 0.3732 - val_loss: 1.9644 - val_acc: 0.4646
Epoch 2/300
782/782 [==============================] - 50s 64ms/step - loss: 1.8273 - acc: 0.5090 - val_loss: 1.7261 - val_acc: 0.5505
Epoch 3/300
782/782 [==============================] - 50s 64ms/step - loss: 1.6065 - acc: 0.5910 - val_loss: 1.5225 - val_acc: 0.6326
Epoch 4/300
782/782 [==============================] - 50s 65ms/step - loss: 1.4187 - acc: 0.6583 - val_loss: 1.4089 - val_acc: 0.6716
Epoch 5/300
782/782 [==============================] - 51s 65ms/step - loss: 1.2839 - acc: 0.7074 - val_loss: 1.2639 - val_acc: 0.7213
Epoch 6/300
782/782 [==============================] - 51s 65ms/step - loss: 1.1844 - acc: 0.7415 - val_loss: 1.2032 - val_acc: 0.7379
Epoch 7/300
782/782 [==============================] - 51s 65ms/step - loss: 1.1051 - acc: 0.7662 - val_loss: 1.1276 - val_acc: 0.7609
Epoch 8/300
782/782 [==============================] - 

782/782 [==============================] - 52s 66ms/step - loss: 0.1958 - acc: 0.9979 - val_loss: 0.5561 - val_acc: 0.9148
Epoch 122/300
782/782 [==============================] - 52s 66ms/step - loss: 0.1948 - acc: 0.9982 - val_loss: 0.5614 - val_acc: 0.9157
Epoch 123/300
782/782 [==============================] - 52s 66ms/step - loss: 0.1941 - acc: 0.9981 - val_loss: 0.5649 - val_acc: 0.9141
Epoch 124/300
782/782 [==============================] - 52s 66ms/step - loss: 0.1936 - acc: 0.9981 - val_loss: 0.5583 - val_acc: 0.9154
Epoch 125/300
782/782 [==============================] - 52s 66ms/step - loss: 0.1919 - acc: 0.9986 - val_loss: 0.5649 - val_acc: 0.9156
Epoch 126/300
782/782 [==============================] - 51s 66ms/step - loss: 0.1917 - acc: 0.9983 - val_loss: 0.5629 - val_acc: 0.9148
Epoch 127/300
782/782 [==============================] - 51s 66ms/step - loss: 0.1915 - acc: 0.9982 - val_loss: 0.5668 - val_acc: 0.9158
Epoch 128/300
782/782 [==============================] 

Epoch 181/300
782/782 [==============================] - 51s 66ms/step - loss: 0.1582 - acc: 0.9994 - val_loss: 0.5671 - val_acc: 0.9160
Epoch 182/300
782/782 [==============================] - 51s 66ms/step - loss: 0.1575 - acc: 0.9995 - val_loss: 0.5771 - val_acc: 0.9166
Epoch 183/300
782/782 [==============================] - 52s 66ms/step - loss: 0.1571 - acc: 0.9995 - val_loss: 0.5843 - val_acc: 0.9134
Epoch 184/300
782/782 [==============================] - 52s 67ms/step - loss: 0.1569 - acc: 0.9994 - val_loss: 0.5758 - val_acc: 0.9154
Epoch 185/300
782/782 [==============================] - 51s 66ms/step - loss: 0.1560 - acc: 0.9995 - val_loss: 0.5638 - val_acc: 0.9148
Epoch 186/300
782/782 [==============================] - 51s 66ms/step - loss: 0.1559 - acc: 0.9991 - val_loss: 0.5672 - val_acc: 0.9155
Epoch 187/300
782/782 [==============================] - 52s 66ms/step - loss: 0.1551 - acc: 0.9995 - val_loss: 0.5716 - val_acc: 0.9163
Epoch 188/300
782/782 [==================

Epoch 300/300
782/782 [==============================] - 51s 66ms/step - loss: 0.1438 - acc: 0.9995 - val_loss: 0.5703 - val_acc: 0.9170
